In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import concurrent.futures
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC


c:\Users\joonw\anaconda3\envs\jl2815\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
df = pd.read_csv("C:\\Users\\joonw\\trav\\trav_dataset1.csv")

from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, stratify=df['convert_ind'], random_state=24)

train_set = train_set.drop(columns=['split'])
test_set = test_set.drop(columns=['split'])

# Separate features and target from the entire training set
y_train = train_set['convert_ind'].values
train_x = train_set.drop(columns=['convert_ind'])

y_test = test_set['convert_ind'].values
test_x = test_set.drop(columns=['convert_ind'])

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(train_x)
X_test = scaler.transform(test_x)

# Reshape the data for Conv1D (add a channel dimension)
X_train = X_train[:, :, np.newaxis]
X_test = X_test[:, :, np.newaxis]

In [5]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))
class_weights


early_stopping = EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True, mode='max')


In [29]:
# Build the CNN model
model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    BatchNormalization(),
    Dropout(0.3),
    
    Conv1D(64, kernel_size=3, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Conv1D(128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Compile the model
# if model is not learning, reduce learning_rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=[AUC(name='auc')])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=25,
    batch_size=32,
    verbose=1,
    class_weight=class_weights,
    callbacks=[early_stopping]
)

Epoch 1/25
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - auc: 0.5436 - loss: 0.9081 - val_auc: 0.6031 - val_loss: 0.7364
Epoch 2/25
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - auc: 0.5620 - loss: 0.7432 - val_auc: 0.6022 - val_loss: 0.6898
Epoch 3/25
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - auc: 0.5801 - loss: 0.7141 - val_auc: 0.6032 - val_loss: 0.6837
Epoch 4/25
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - auc: 0.5887 - loss: 0.6964 - val_auc: 0.6121 - val_loss: 0.6846
Epoch 5/25
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - auc: 0.6135 - loss: 0.6791 - val_auc: 0.6305 - val_loss: 0.6895
Epoch 6/25
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - auc: 0.6095 - loss: 0.6803 - val_auc: 0.6357 - val_loss: 0.7044
Epoch 7/25
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - auc: 0.6243 - loss: 0.6711 - val_auc: 0.6415 - val_loss: 0.7208
Epoch 8/25
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - auc: 0.6303 - loss: 0.6782 - val_auc: 0.6388 - val_loss: 0.6967
Epoch 9/25
1230/1230 ━━━

In [30]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")

# Make predictions
predictions = model.predict(X_test)

# Calculate AUC
auc = roc_auc_score(y_test, predictions)
print(f"Test AUC: {auc:.4f}")


Test Accuracy: 0.6519
308/308 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
Test AUC: 0.6517


## Tabnet

### !pip install pytorch-tabnet

Tabnet requires np array

In [10]:
# Split the data
train_set, test_set = train_test_split(df, test_size=0.2, stratify=df['convert_ind'], random_state=24)

# Drop the 'split' column if it exists
if 'split' in train_set.columns:
    train_set = train_set.drop(columns=['split'])
    test_set = test_set.drop(columns=['split'])

# Separate features and target
y_train = train_set['convert_ind'].values
train_x = train_set.drop(columns=['convert_ind'])

y_test = test_set['convert_ind'].values
test_x = test_set.drop(columns=['convert_ind'])

# Select numeric columns
train_x = train_x.select_dtypes(include=[np.number])
test_x = test_x.select_dtypes(include=[np.number])

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(train_x)
X_test = scaler.transform(test_x)

# Ensure inputs are NumPy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)



Hyper parameter optimization

In [ ]:

from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score

best_auc = 0
best_params = {}

# Parameter ranges
param_grid = {
    'n_d': [8, 16, 32],
    'n_a': [8, 16, 32],
    'n_steps': [3, 5, 7],
    'gamma': [1.0, 1.5, 2.0],
    'lambda_sparse': [1e-4, 1e-3, 1e-2],
    'learning_rate': [0.01, 0.05, 0.1],
}

# Manual loop for tuning
for n_d in param_grid['n_d']:
    for n_a in param_grid['n_a']:
        for n_steps in param_grid['n_steps']:
            for gamma in param_grid['gamma']:
                for lambda_sparse in param_grid['lambda_sparse']:
                    for lr in param_grid['learning_rate']:
                        # Define the model
                        model = TabNetClassifier(
                            n_d=n_d,
                            n_a=n_a,
                            n_steps=n_steps,
                            gamma=gamma,
                            lambda_sparse=lambda_sparse,
                            optimizer_params=dict(lr=lr),
                        )
                        
                        # Train the model
                        model.fit(
                            X_train, y_train,
                            eval_set=[(X_test, y_test)],
                            eval_metric=['auc'],
                            max_epochs=100,
                            batch_size=128,
                            patience=10
                        )
                        
                        # Evaluate the model
                        preds = model.predict_proba(X_test)[:, 1]
                        auc = roc_auc_score(y_test, preds)
                        print(f"Params: n_d={n_d}, n_a={n_a}, n_steps={n_steps}, gamma={gamma}, lambda_sparse={lambda_sparse}, lr={lr}, AUC={auc:.4f}")

                        # Track the best parameters
                        if auc > best_auc:
                            best_auc = auc
                            best_params = {
                                'n_d': n_d,
                                'n_a': n_a,
                                'n_steps': n_steps,
                                'gamma': gamma,
                                'lambda_sparse': lambda_sparse,
                                'learning_rate': lr,
                            }

print("Best AUC:", best_auc)
print("Best Parameters:", best_params)


c:\Users\joonw\anaconda3\envs\jl2815\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.29552 | val_0_auc: 0.56489 |  0:01:22s
epoch 1  | loss: 0.28579 | val_0_auc: 0.59315 |  0:02:28s
epoch 2  | loss: 0.28362 | val_0_auc: 0.62833 |  0:03:33s
epoch 3  | loss: 0.28237 | val_0_auc: 0.61049 |  0:04:37s
epoch 4  | loss: 0.2809  | val_0_auc: 0.61878 |  0:05:43s
epoch 5  | loss: 0.28116 | val_0_auc: 0.62705 |  0:06:52s
epoch 6  | loss: 0.28051 | val_0_auc: 0.62824 |  0:08:05s
epoch 7  | loss: 0.28004 | val_0_auc: 0.62607 |  0:09:14s
epoch 8  | loss: 0.28055 | val_0_auc: 0.62687 |  0:10:20s
epoch 9  | loss: 0.27957 | val_0_auc: 0.63426 |  0:11:25s
epoch 10 | loss: 0.27825 | val_0_auc: 0.62437 |  0:12:32s
epoch 11 | loss: 0.27945 | val_0_auc: 0.62592 |  0:13:37s
epoch 12 | loss: 0.27894 | val_0_auc: 0.62065 |  0:14:42s
epoch 13 | loss: 0.27938 | val_0_auc: 0.61556 |  0:15:50s
epoch 14 | loss: 0.27887 | val_0_auc: 0.6284  |  0:16:56s
epoch 15 | loss: 0.27932 | val_0_auc: 0.6303  |  0:18:01s
epoch 16 | loss: 0.27851 | val_0_auc: 0.6318  |  0:19:08s
epoch 17 | los

c:\Users\joonw\anaconda3\envs\jl2815\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Params: n_d=8, n_a=8, n_steps=3, gamma=1.0, lambda_sparse=0.0001, lr=0.01, AUC=0.6343


c:\Users\joonw\anaconda3\envs\jl2815\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


...

epoch 0  | loss: 0.2925  | val_0_auc: 0.60837 |  0:01:04s
epoch 1  | loss: 0.28663 | val_0_auc: 0.60076 |  0:02:07s
epoch 2  | loss: 0.28391 | val_0_auc: 0.60166 |  0:03:12s
epoch 3  | loss: 0.28468 | val_0_auc: 0.61228 |  0:04:13s
epoch 4  | loss: 0.28361 | val_0_auc: 0.60704 |  0:05:18s
epoch 5  | loss: 0.28384 | val_0_auc: 0.62126 |  0:06:19s
epoch 6  | loss: 0.283   | val_0_auc: 0.6279  |  0:07:18s
epoch 7  | loss: 0.28327 | val_0_auc: 0.63156 |  0:08:24s
epoch 8  | loss: 0.28312 | val_0_auc: 0.63224 |  0:09:24s
epoch 9  | loss: 0.28184 | val_0_auc: 0.62674 |  0:10:24s
epoch 10 | loss: 0.28121 | val_0_auc: 0.61855 |  0:11:26s
epoch 11 | loss: 0.28003 | val_0_auc: 0.62582 |  0:12:27s
epoch 12 | loss: 0.28007 | val_0_auc: 0.6231  |  0:13:27s
epoch 13 | loss: 0.28053 | val_0_auc: 0.6438  |  0:14:42s
epoch 14 | loss: 0.27925 | val_0_auc: 0.64049 |  0:15:49s
epoch 15 | loss: 0.28027 | val_0_auc: 0.63901 |  0:16:57s
epoch 16 | loss: 0.27948 | val_0_auc: 0.64428 |  0:18:05s
epoch 17 | los

c:\Users\joonw\anaconda3\envs\jl2815\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Params: n_d=8, n_a=8, n_steps=3, gamma=1.5, lambda_sparse=0.0001, lr=0.05, AUC=0.6629


c:\Users\joonw\anaconda3\envs\jl2815\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.29491 | val_0_auc: 0.58021 |  0:01:04s
epoch 1  | loss: 0.29021 | val_0_auc: 0.55617 |  0:02:07s
epoch 2  | loss: 0.29039 | val_0_auc: 0.55875 |  0:03:16s
epoch 3  | loss: 0.28932 | val_0_auc: 0.58041 |  0:04:29s
epoch 4  | loss: 0.28831 | val_0_auc: 0.61111 |  0:05:31s
epoch 5  | loss: 0.28703 | val_0_auc: 0.60974 |  0:06:33s


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")

# Make predictions
predictions = model.predict(X_test)

# Calculate AUC
auc = roc_auc_score(y_test, predictions)
print(f"Test AUC: {auc:.4f}")